In [1]:
#建议改为trainningmodel

# 定义集成学习器

In [2]:
from sklearn.base import BaseEstimator, RegressorMixin, clone
    #sklearn.base.clone的功能为使用相同的参数构造一个新的不拟合估计量。
    
class EnsembleLearners(BaseEstimator, RegressorMixin):
    def __init__(self, models, weight):
        self.models = models
        self.weight = weight

    def fit(self, X, y): 
        self.models_ = [clone(x) for x in self.models]
        for model in self.models_:
            model.fit(X, y)
        return self

    def predict(self, X): 
        w = list()
        pred = np.array([model.predict(X) for model in self.models_])
        
        for data in range(pred.shape[1]):
            single = [pred[model, data] * weight for model, weight in zip(range(pred.shape[0]), self.weight)]
            w.append(np.sum(single))
        return w

# 参数设置

In [3]:
#原函数头部
# def modelTrain(df,LOG_N,gbr_EST,gbr_FEAT,gbr_LRAT,etr_EST,etr_FEAT,
    #rf_EST,rf_FEAT,xgb_EST,xgb_SAM,xgb_LRAT,AlgorithmsInEL):
   

In [4]:
import pandas as pd
df_bank= pd.read_csv ("Data/df_bank.csv", index_col=0)
df=df_bank
LOG_N=10
etr_EST = 25
etr_FEAT = 0.8
gbr_EST = 200
gbr_FEAT = 0.8
gbr_LRAT = 0.1
rf_EST = 25
rf_FEAT = 0.8
xgb_EST = 300
xgb_SAM = 0.8
xgb_LRAT =0.1
AlgorithmsInEL=2

In [5]:
df_bank.columns

Index(['TICKER_SYMBOL', 'END_DATE', 'C_RESER_CB', 'RANK_C_RESER_CB',
       'RANK_REVENUE_Q1', 'REVENUE_Q1', 'T_LIAB',
       'RANK_T_LIAB_FINAN_LEASE_RECEIV', 'RANK_T_ASSETS_FINAN_LEASE_RECEIV',
       'RANK_T_LIAB',
       ...
       'T_ASSETS_AVAIL_FOR_SALE_FA', 'DIS_PREFERRED_STOCK_E_GROWTH6',
       'T_SH_EQUITY_MINORITY_INT_LAG3', 'DATA_VALUE2170002035',
       'DISBUR_LA_GROWTH12_LAG1', 'T_SH_EQUITY_LAG4',
       'T_SH_EQUITY_DERIV_LIAB_LAG2', 'RANK_SPEC_OR_Q1', 'AE_GROWTH9_LAG2',
       'target_REVENUE_Q1'],
      dtype='object', length=2619)

In [6]:
#将 'END_DATE'为日期类型
df.END_DATE = df['END_DATE'].astype("datetime64[ns]")

In [7]:
if(AlgorithmsInEL==0):
    df = df.loc[df['END_DATE'].dt.month == 3,]

In [8]:
#缺失值处理    
import numpy as np
df = df.replace([np.inf, -np.inf], np.nan)

# 测试集与训练集的划分

In [9]:
#查看列名
df.columns

Index(['TICKER_SYMBOL', 'END_DATE', 'C_RESER_CB', 'RANK_C_RESER_CB',
       'RANK_REVENUE_Q1', 'REVENUE_Q1', 'T_LIAB',
       'RANK_T_LIAB_FINAN_LEASE_RECEIV', 'RANK_T_ASSETS_FINAN_LEASE_RECEIV',
       'RANK_T_LIAB',
       ...
       'T_ASSETS_AVAIL_FOR_SALE_FA', 'DIS_PREFERRED_STOCK_E_GROWTH6',
       'T_SH_EQUITY_MINORITY_INT_LAG3', 'DATA_VALUE2170002035',
       'DISBUR_LA_GROWTH12_LAG1', 'T_SH_EQUITY_LAG4',
       'T_SH_EQUITY_DERIV_LIAB_LAG2', 'RANK_SPEC_OR_Q1', 'AE_GROWTH9_LAG2',
       'target_REVENUE_Q1'],
      dtype='object', length=2619)

In [10]:
#查看因变量
df['target_REVENUE_Q1']

0      3.760604e+09
1      3.730246e+09
2      3.623103e+09
3      4.000487e+09
4      4.084252e+09
           ...     
766    7.419590e+08
767    6.896060e+08
768    6.723730e+08
769    6.982370e+08
770             NaN
Name: target_REVENUE_Q1, Length: 768, dtype: float64

In [11]:
#测试集的生成
    #测试集的截止日期为2018-03-31
    #END_DATE	End date	截止日期

X_test = df.loc[(df['target_REVENUE_Q1'].isnull()) & (df['END_DATE'] == '2018-03-31'), ]
X_test.drop('target_REVENUE_Q1', axis=1, inplace=True)

C:\Users\soloman\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [12]:
#训练集的产生
X = df.loc[(df['target_REVENUE_Q1'] > 0.), ]
y = X['target_REVENUE_Q1']
X.drop('target_REVENUE_Q1', axis=1, inplace=True)

X.loc[:, X.dtypes == 'float64'] = X.loc[:, X.dtypes == "float64"].fillna(0.00001)
cols = [i for i in df.columns if i not in ['TICKER_SYMBOL', 'END_DATE', 'target_REVENUE_Q1']]
X = X[cols]

y = y / 10000.0
log_y = np.log(y)/np.log(LOG_N)

C:\Users\soloman\AppData\Roaming\Python\Python37\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [13]:
#数据标准化处理
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(df[cols])

MinMaxScaler(copy=True, feature_range=(0, 1))

# 模型训练

In [14]:
#创建四种模型的原型
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from xgboost import XGBRegressor

xgb = XGBRegressor(n_estimators = xgb_EST, subsample=xgb_SAM, learning_rate = xgb_LRAT)
extra = ExtraTreesRegressor(n_estimators = etr_EST, max_features= etr_FEAT)
gbr = GradientBoostingRegressor(n_estimators = gbr_EST, max_features= gbr_FEAT, learning_rate = gbr_LRAT)
rf = RandomForestRegressor(n_estimators = rf_EST, max_features= rf_FEAT)

In [15]:
# 采用集成学习方法
    #根据AlgorithmsInEL的值，选择不同的模型，计算平均值

if(AlgorithmsInEL == 0):
    print([extra,rf])
    ensembleModel = EnsembleLearners(models=[extra,rf], weight= [0.5,0.5])
elif(AlgorithmsInEL == 1):
    print([gbr,xgb])
    ensembleModel = EnsembleLearners(models=[gbr,xgb], weight= [0.5,0.5])
elif(AlgorithmsInEL == 2):
    print([extra,gbr,rf,xgb])
    ensembleModel = EnsembleLearners(models=[extra,gbr,rf,xgb], weight= [0.25,0.25,0.25,0.25])

[ExtraTreesRegressor(bootstrap=False, criterion='mse', max_depth=None,
                    max_features=0.8, max_leaf_nodes=None,
                    min_impurity_decrease=0.0, min_impurity_split=None,
                    min_samples_leaf=1, min_samples_split=2,
                    min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=None,
                    oob_score=False, random_state=None, verbose=0,
                    warm_start=False), GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=0.8, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=200,
                          n_iter_no_change=None, presort='auto',
                          random_state=None, subsample=1.0,

In [16]:
#标准化处理
X_scaled = scaler.transform(X)

In [17]:
#拟合
ensembleModel.fit(X_scaled,log_y)

EnsembleLearners(models=[ExtraTreesRegressor(bootstrap=False, criterion='mse',
                                             max_depth=None, max_features=0.8,
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators=25, n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
                         GradientBoost...
                                      learning_rate=0.1, max_delta_step=None,
                                      max_depth=None, min_child_weight=None,


# 模型预测

In [18]:
#在测试集上预测
to_pred_x = X_test[cols]
to_pred_scaled_x = scaler.transform(to_pred_x)
pred = np.power(LOG_N, ensembleModel.predict(to_pred_scaled_x)) * 10000.

submit_symbols = list(X_test.TICKER_SYMBOL)
submit_results = list((np.array(X_test.REVENUE_Q1) + np.array(pred)) / 1000000.)
submit_results = [float('%.2f' % w) for w in submit_results]

res = pd.DataFrame({'TICKER_SYMBOL': submit_symbols, 'PRED': submit_results})
res.head()

,TICKER_SYMBOL,PRED
0,1,56026.49
1,2142,13306.80
2,2807,1278.13
3,2839,1271.60
4,600000,78017.10
